# Data sourcing

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import matplotlib
%matplotlib inline
import numpy as np
import pandas as pd

## 1️⃣ CSV

In [4]:
tracks_df = pd.read_csv('data/spotify_2017.csv')
tracks_df[['name', 'artists']].head()

,name,artists
0,Shape of You,Ed Sheeran
1,Despacito - Remix,Luis Fonsi
2,Despacito (Featuring Daddy Yankee),Luis Fonsi
3,Something Just Like This,The Chainsmokers
4,I'm the One,DJ Khaled


## 2️⃣ API

In [5]:
from music import fetch_lyrics

In [6]:
fetch_lyrics("Ed Sheeran", "Shape of Me")

'No Data'

In [7]:
fetch_lyrics("Ed Sheeran", "Shape of You")[0:100]

"The club isn't the best place to find a lover\nSo the bar is where I go (mmmm)\nMe and my friends at t"

In [8]:
%%time
# use iterrows to iterate, it's a kind of enumerate
for index, row in tracks_df.iterrows():
    print(index, row['name'], row['artists'])
    lyrics = fetch_lyrics(row['artists'], row['name'])
    # print(lyrics[0:10])
    tracks_df.loc[index, 'lyrics'] = lyrics

0 Shape of You Ed Sheeran
1 Despacito - Remix Luis Fonsi
2 Despacito (Featuring Daddy Yankee) Luis Fonsi
3 Something Just Like This The Chainsmokers
4 I'm the One DJ Khaled
5 HUMBLE. Kendrick Lamar
6 It Ain't Me (with Selena Gomez) Kygo
7 Unforgettable French Montana
8 That's What I Like Bruno Mars
9 I Don’t Wanna Live Forever (Fifty Shades Darker) - From "Fifty Shades Darker (Original Motion Picture Soundtrack)" ZAYN
10 XO TOUR Llif3 Lil Uzi Vert
11 Paris The Chainsmokers
12 Stay (with Alessia Cara) Zedd
13 Attention Charlie Puth
14 Mask Off Future
15 Congratulations Post Malone
16 Swalla (feat. Nicki Minaj & Ty Dolla $ign) Jason Derulo
17 Castle on the Hill Ed Sheeran
18 Rockabye (feat. Sean Paul & Anne-Marie) Clean Bandit
19 Believer Imagine Dragons
20 Mi Gente J Balvin
21 Thunder Imagine Dragons
22 Say You Won't Let Go James Arthur
23 There's Nothing Holdin' Me Back Shawn Mendes
24 Me Rehúso Danny Ocean
25 Issues Julia Michaels
26 Galway Girl Ed Sheeran
27 Scared to Be Lonely Marti

In [9]:
tracks_df.head()

,id,name,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,lyrics
0,7qiZfU4dY1lWllzX7mPBI,Shape of You,Ed Sheeran,0.825,0.652,1.0,-3.183,0.0,0.0802,0.5810,0.000000,0.0931,0.931,95.977,233713.0,4.0,The club isn't the best place to find a lover\...
1,5CtI0qwDJkDQGwXD1H1cL,Despacito - Remix,Luis Fonsi,0.694,0.815,2.0,-4.328,1.0,0.1200,0.2290,0.000000,0.0924,0.813,88.931,228827.0,4.0,No Data
2,4aWmUDTfIPGksMNLV2rQP,Despacito (Featuring Daddy Yankee),Luis Fonsi,0.660,0.786,2.0,-4.757,1.0,0.1700,0.2090,0.000000,0.1120,0.846,177.833,228200.0,4.0,No Data
3,6RUKPb4LETWmmr3iAEQkt,Something Just Like This,The Chainsmokers,0.617,0.635,11.0,-6.769,0.0,0.0317,0.0498,0.000014,0.1640,0.446,103.019,247160.0,4.0,I've been reading books of old\nThe legends an...
4,3DXncPQOG4VBw3QHh3S81,I'm the One,DJ Khaled,0.609,0.668,7.0,-4.284,1.0,0.0367,0.0552,0.000000,0.1670,0.811,80.924,288600.0,4.0,[DJ Khaled:]\nWe The Best Music\nAnother One!\...


## 3️⃣ SQL

In [10]:
import sqlite3

conn = sqlite3.connect('data/music.sqlite')

In [11]:
# c = conn.cursor()

In [12]:
limit = 500000
query = """
    SELECT ai.artist_mb, p.listeners_lastfm 
    FROM artist_info ai 
    JOIN popularity p ON ai.mbid = p.mbid
    WHERE p.listeners_lastfm > ?
"""

In [13]:
# c.execute(query, (limit,))
# rows = c.fetchall()
# rows[0:10]

In [14]:
listens_df = pd.read_sql(query, conn, params=(limit,))
listens_df.head()

,artist_mb,listeners_lastfm
0,Coldplay,5381567.0
1,Radiohead,4732528.0
2,Red Hot Chili Peppers,4620835.0
3,Rihanna,4558193.0
4,Eminem,4517997.0


In [15]:
tracks_df = tracks_df.merge(
    listens_df,
    left_on='artists',
    right_on='artist_mb',
    how='left'
)

In [16]:
tracks_df.head()

,id,name,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,lyrics,artist_mb,listeners_lastfm
0,7qiZfU4dY1lWllzX7mPBI,Shape of You,Ed Sheeran,0.825,0.652,1.0,-3.183,0.0,0.0802,0.5810,0.000000,0.0931,0.931,95.977,233713.0,4.0,The club isn't the best place to find a lover\...,Ed Sheeran,1620379.0
1,5CtI0qwDJkDQGwXD1H1cL,Despacito - Remix,Luis Fonsi,0.694,0.815,2.0,-4.328,1.0,0.1200,0.2290,0.000000,0.0924,0.813,88.931,228827.0,4.0,No Data,NaN,NaN
2,4aWmUDTfIPGksMNLV2rQP,Despacito (Featuring Daddy Yankee),Luis Fonsi,0.660,0.786,2.0,-4.757,1.0,0.1700,0.2090,0.000000,0.1120,0.846,177.833,228200.0,4.0,No Data,NaN,NaN
3,6RUKPb4LETWmmr3iAEQkt,Something Just Like This,The Chainsmokers,0.617,0.635,11.0,-6.769,0.0,0.0317,0.0498,0.000014,0.1640,0.446,103.019,247160.0,4.0,I've been reading books of old\nThe legends an...,The Chainsmokers,794664.0
4,3DXncPQOG4VBw3QHh3S81,I'm the One,DJ Khaled,0.609,0.668,7.0,-4.284,1.0,0.0367,0.0552,0.000000,0.1670,0.811,80.924,288600.0,4.0,[DJ Khaled:]\nWe The Best Music\nAnother One!\...,DJ Khaled,1160535.0


## 4️⃣ Google BigQuery

In [ ]:
!pip install --quiet pandas-gbq

In [ ]:
import pandas_gbq

In [ ]:
project_id = 'lwt-data-lectures' # TODO: replace with your own!

sql = """
SELECT artist_name, COUNT(artist_name) FROM `listenbrainz.listenbrainz.listen` 
WHERE listened_at BETWEEN "2017-01-01" AND "2018-01-01"
GROUP BY artist_name
HAVING COUNT(artist_name) > 1000
ORDER BY COUNT(artist_name) DESC
"""
music_brainz_df = pandas_gbq.read_gbq(sql, project_id=project_id)

In [ ]:
music_brainz_df

In [ ]:
music_brainz_df.columns = ["artists", "music_brainz_plays"]

In [ ]:
music_brainz_df.head()

In [ ]:
tracks_df = tracks_df.merge(
    music_brainz_df,
    on='artists',
    how='left'
)

In [ ]:
tracks_df.head()

## 5️⃣ Scraping

In [ ]:
from bs4 import BeautifulSoup
import re
import requests

In [ ]:
def scrape_bday(artist):
    artist = artist.replace(" ", "_")
    url = f'https://en.wikipedia.org/wiki/{artist}'
    try:
        response = requests.get(url)
        raw_html = response.content
        soup = BeautifulSoup(raw_html, 'html.parser')
        bday = soup.find('span', class_='bday').text
        return bday
    except:
        return 'Inconclusive'

In [ ]:
scrape_bday("Ed Sheera")

In [ ]:
scrape_bday("Ed Sheeran")

In [ ]:
artists_list = list(set(tracks_df["artists"].tolist()))
len(artists_list)

In [ ]:
birthdays = []
for artist in artists_list:
    birthday = scrape_bday(artist)
    birthdays.append(birthday) 

In [ ]:
birthdays[0:10]

In [ ]:
birthdays_df = pd.DataFrame({"artists": artists_list,
                            "birthday": birthdays})

In [ ]:
birthdays_df.head()

In [ ]:
tracks_df = tracks_df.merge(
    birthdays_df,
    on='artists',
    how='left'
)

In [ ]:
tracks_df.head()

In [ ]:
tracks_df["music_brainz_plays"].hist(bins = 20);

In [ ]:
tracks_df["listeners_lastfm"].hist(bins = 20);

In [ ]:
import re
pattern = r'\d{4}-\d{2}-\d{2}'
# Find only the rows that have valid birthdays
only_bdays = tracks_df[tracks_df["birthday"].str.match(pattern)].copy()
# Convert to a datetime format
tracks_df["birthday"] = pd.to_datetime(only_bdays["birthday"])

In [ ]:
tracks_df.plot.scatter("birthday", "listeners_lastfm")
